<a href="https://colab.research.google.com/github/eyaler/avatars4all/blob/master/yarok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video Green Screen
## https://github.com/NathanUA/U-2-Net
## https://github.com/ZHKKKe/MODNet
### Made just a little bit more accessible by Eyal Gruss (https://eyalgruss.com, eyalgruss@gmail.com)

#### Foreground options:
*   Image from web or uploaded
*   Video from web or uploaded
*   Mirrored versions of the above

#### Model options:
*   U^2-Net
*   MODNet photographic
*   MODNet webcam

#### Mask options:
*   Continuous blending
*   One frame delay smoothing with custom threshold

#### Background options:
*   Image from web or uploaded
*   Video from web or uploaded
*   Mirrored versions of the above
*   Solid black, white or green
*   Trasparent (for images)
*   Foreground converted to grayscale

##### List of more generative tools: https://j.mp/generativetools



In [ ]:
#@title Setup
%cd /content
!git clone --depth 1 https://github.com/NathanUA/U-2-Net
!mkdir -p /content/U-2-Net/saved_models/u2net
%cd /content/U-2-Net/saved_models/u2net
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/u2net.pth
#!wget --no-check-certificate -nc https://eyalgruss.com/fomm/u2netp.pth
#!wget --no-check-certificate -nc https://eyalgruss.com/fomm/u2net_portrait.pth
%cd /content
!git clone --depth 1 https://github.com/ZHKKKe/MODNet
%cd MODNet/pretrained
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/modnet_photographic_portrait_matting.ckpt
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/modnet_webcam_portrait_matting.ckpt
%cd /content
!pip install -U youtube-dl
!pip install -U imageio
!pip install -U imageio-ffmpeg

In [ ]:
#@title Get the foreground image/video and background image/video from the web
#@markdown 1. You can change the URLs to your **own** stuff!
#@markdown 2. For the background you can use the drop-down menu to alternatively choose a **solid color**, **transparent** (for images only) or a **greyscale** version of the foreground
#@markdown 3. Alternatively, you can upload **local** files in the next cells

foreground_url = 'https://www.youtube.com/watch?v=kMpnwIGDQvU' #@param {type:"string"}
background_url = 'https://www.youtube.com/watch?v=dMvnCyznteU' #@param ['Black', 'White', 'Green', 'Transparent', 'Gray foreground'] {allow-input: true}

import os
import youtube_dl
def is_supported(url):
    if url.lower().endswith(('.png','.jpg','.jpeg','.bmp')):
      return False
    extractors = youtube_dl.extractor.gen_extractors()
    for e in extractors:
        if e.suitable(url) and e.IE_NAME != 'generic':
            return True
    return False

if foreground_url:
  !rm -f /content/foreground
  if is_supported(foreground_url):
    !rm -f /content/foreground.mp4
    !youtube-dl -f 'bestvideo[ext=mp4][vcodec!*=av01]+bestaudio[ext=m4a]/mp4' '$foreground_url' --merge-output-format mp4 -o /content/foreground
    !mv /content/foreground.mp4 /content/foreground 
    fg_time_params = ''
  if not os.path.exists('/content/foreground'):
    !wget '$foreground_url' -O /content/foreground

if '://' in background_url:
  !rm -f /content/background
  if is_supported(background_url):
    !rm -f /content/background.mp4
    !youtube-dl -f 'bestvideo[ext=mp4][vcodec!*=av01]+bestaudio[ext=m4a]/mp4' '$background_url' --merge-output-format mp4 -o /content/background
    !mv /content/background.mp4 /content/background
    bg_time_params = ''
  if not os.path.exists('/content/background'):
    !wget '$background_url' -O /content/background

In [ ]:
#@title Optionally upload foreground image/video { run: "auto" }
manually_upload_foreground = False #@param {type:"boolean"}
if manually_upload_foreground:
  from google.colab import files
  import shutil

  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/foreground')
    break
  foreground_url = None
  fg_time_params = ''

In [ ]:
#@title Optionally upload local background image/video { run: "auto" }
manually_upload_background = False #@param {type:"boolean"}
if manually_upload_background:
  from google.colab import files
  import shutil

  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/background')
    break
  background_url = None
  bg_time_params = ''

In [ ]:
#@title Optionally shorten foreground video
start_seconds =  0#@param {type:"number"}
duration_seconds =  60#@param {type:"number"}
start_seconds = max(start_seconds,0)
duration_seconds = max(duration_seconds,0)
fg_time_params = '-ss %i -t %i'%(start_seconds, duration_seconds)

In [ ]:
#@title Optionally shorten background video
start_seconds =  0#@param {type:"number"}
duration_seconds =  60#@param {type:"number"}
start_seconds = max(start_seconds,0)
duration_seconds = max(duration_seconds,0)
bg_time_params = '-ss %i -t %i'%(start_seconds, duration_seconds)

In [ ]:
#@title Green Screen It!

model = 'u2net' #@param ['u2net', 'modnet_photographic', 'modnet_webcam']
#model = 'u2net' #@param ['u2net', 'u2netp', 'u2net_portrait', 'modnet_photographic', 'modnet_webcam']
one_frame_delay = True #@param {type:"boolean"} 
one_frame_delay_threshold = 0.1 #@param {type:"slider", min:0, max:1, step:0.05}
mirror_foreground = False #@param {type:"boolean"} 
mirror_background = False #@param {type:"boolean"} 
retain_aspect_background = True #@param {type:"boolean"}
copy_audio = True #@param {type:"boolean"}

fg_dir = '/content/U-2-Net/test_data/test_images'
mask_dir = '/content/U-2-Net/test_data/u2net_results'
bg_dir = '/content/bg_frames'
out_dir = '/content/out_frames'
!rm -rf $fg_dir
!mkdir -p $fg_dir
!rm -rf $mask_dir
!mkdir -p $mask_dir
!rm -rf $bg_dir
!mkdir -p $bg_dir
!rm -rf $out_dir
!mkdir -p $out_dir

import imageio
import cv2
import numpy as np
import os
from time import time
import io
import PIL
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from MODNet.src.models.modnet import MODNet
import warnings
warnings.filterwarnings("ignore")

torch_transforms = transforms.Compose(
  [
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
  ]
)

def fix_dims(im):
    if im.ndim == 2:
        im = np.tile(im[..., None], [1, 1, 3])
    return im[...,:3]

def crop_resize(im, size, crop=False):
  if im.shape[:2] == size:
    return im
  if size[0]<im.shape[0] or size[1]<im.shape[1]:
    interp = cv2.INTER_AREA
  else:
    interp = cv2.INTER_CUBIC
  if not crop:
    return np.clip(cv2.resize(im, size[::-1], interpolation=interp),0,1)
  ratio = max(size[0]/im.shape[0], size[1]/im.shape[1])
  im = np.clip(cv2.resize(im, (int(np.ceil(im.shape[1]*ratio)), int(np.ceil(im.shape[0]*ratio))), interpolation=interp),0,1)
  return im[(im.shape[0]-size[0])//2:(im.shape[0]-size[0])//2+size[0], (im.shape[1]-size[1])//2:(im.shape[1]-size[1])//2+size[1]]

ref_size = 512
def modnet_matting(modnet, im):
  im_h, im_w = im.shape[:2]
  im_tensor = torch_transforms(im).float()
  im_tensor = im_tensor[None, :, :, :].cuda()
  
  if max(im_h, im_w) < ref_size or min(im_h, im_w) > ref_size:
    if im_w >= im_h:
      im_rh = ref_size
      im_rw = int(im_w / im_h * ref_size)
    elif im_w < im_h:
      im_rw = ref_size
      im_rh = int(im_h / im_w * ref_size)
  else:
    im_rh = im_h
    im_rw = im_w
        
  im_rw = im_rw - im_rw % 32
  im_rh = im_rh - im_rh % 32
  
  if im_h!=im_rh or im_w!=im_rw:
    im_tensor = F.interpolate(im_tensor, size=(im_rh, im_rw), mode='area')
  
  with torch.no_grad():
    _, _, matte_tensor = modnet(im_tensor, True)
  matte_tensor = F.interpolate(matte_tensor, size=(im_h, im_w), mode='area')
  matte_tensor = matte_tensor.repeat(1, 3, 1, 1)
  return matte_tensor[0].data.cpu().numpy().transpose(1, 2, 0)

grand_start = time()
try:
    fg_now = imageio.imread('/content/foreground')
    fg_now = fix_dims(fg_now)
    imageio.imwrite(fg_dir+'/frame_%05d.png'%1, fg_now)
    fg_now = fg_now/255
except Exception:
    !ffmpeg $fg_time_params -i /content/foreground $fg_dir/frame_%05d.png
    fg = imageio.imread(fg_dir+'/frame_%05d.png'%1)
fg_files = [x for x in sorted(os.listdir(fg_dir)) if x.endswith('.png')]

if background_url == 'Black':
  bg = np.full_like(fg, 0)
elif background_url == 'White':
  bg = np.full_like(fg, 1)
elif background_url == 'Green':
  bg = np.full_like(fg, [0,1,0])
elif background_url not in ('Transparent','Gray foreground'):
  try:
    bg = imageio.imread('/content/background')/255
    bg = fix_dims(bg)
    bg = crop_resize(bg, fg.shape[:2], crop=retain_aspect_background)
    if mirror_background:
      bg = np.fliplr(bg)
  except Exception:
    !ffmpeg $bg_time_params -i /content/background $bg_dir/frame_%05d.png
bg_files = [x for x in sorted(os.listdir(bg_dir)) if x.endswith('.png')]
prepare_time = time()-grand_start

start = time()
if model.startswith('u2net'):
  %cd /content/U-2-Net
  !python /content/U-2-Net/u2net_test.py
elif model.startswith('modnet'):
  %cd /content/MODNet
  modnet = MODNet(backbone_pretrained=False)
  modnet = nn.DataParallel(modnet).cuda()
  modnet.load_state_dict(torch.load('/content/MODNet/pretrained/'+model+'_portrait_matting.ckpt'))
  modnet.eval()
mask_time = time()-start

start = time()
fg_plus = None
mask_plus = None
fg_mode = len(fg_files)>1 or not bg_files
if fg_mode:
  iter_files = fg_files
else:
  iter_files = bg_files
  fg_now = imageio.imread(fg_dir+'/frame_%05d.png'%1)/255
  if model.startswith('u2net'):
    mask = imageio.imread(mask_dir+'/frame_%05d.png'%1)/255
  else:
    mask = modnet_matting(modnet, fg_now)    
  if mirror_foreground:
    fg_now = np.fliplr(fg_now)
    mask = np.fliplr(mask)

for i,file in enumerate(iter_files):
    if fg_mode:
      if one_frame_delay and i>0 and i<len(fg_files)-1:
        fg_now = fg_plus
        mask_minus = mask_now
        mask_now = mask_plus
        if fg_now is None:
          fg_now = imageio.imread(fg_dir+'/'+file)/255
        if model.startswith('u2net'):
          if mask_now is None:
            mask_now = imageio.imread(mask_dir+'/'+file)/255
          mask_plus = imageio.imread(mask_dir+'/'+fg_files[i+1])/255
        else:
          if mask_now is None:
            mask_now = modnet_matting(modnet, fg_now)
          fg_plus = imageio.imread(fg_dir+'/'+fg_files[i+1])/255
          mask_plus = modnet_matting(modnet, fg_plus)
        cond = (np.abs(mask_plus-mask_minus)<=one_frame_delay_threshold) & (np.abs(mask_now-mask_minus)>one_frame_delay_threshold) & (np.abs(mask_now-mask_plus)>one_frame_delay_threshold)
        mask = mask_now*(1-cond) + (mask_minus+mask_plus)/2*cond
      else:
        fg_now = imageio.imread(fg_dir+'/'+file)/255
        if model.startswith('u2net'):
          mask_now = imageio.imread(mask_dir+'/'+file)/255
        else:
          mask_now = modnet_matting(modnet, fg_now)
        mask = mask_now
      if mirror_foreground:
          fg_now = np.fliplr(fg_now)
          mask = np.fliplr(mask)
      if background_url == 'Gray foreground':
        bg = fix_dims(np.dot(fg_now, [0.2989, 0.5870, 0.1140]))
      elif bg_files:
        bg = imageio.imread(bg_dir+'/'+bg_files[i%len(bg_files)])/255
    else:
      bg = imageio.imread(bg_dir+'/'+file)/255
    if bg_files:
      bg = crop_resize(bg, fg_now.shape[:2], crop=retain_aspect_background)
      if mirror_background:
        bg = np.fliplr(bg)
    if background_url == 'Transparent':
      im = np.dstack([fg_now,mask[:,:,0]])
    else:
      im = bg*(1-mask)+fg_now*mask
    imageio.imwrite(out_dir+'/'+file, np.uint8(im*255), compression=0 if len(fg_files)>1 or bg_files else 9)
    print('%d/%d'%(i+1,len(iter_files)))
blend_time = time()-start

start = time()
from IPython.display import HTML, clear_output, Image
from base64 import b64encode
import shutil
!rm -f /content/final.mp4
!rm -f /content/final.png
if len(fg_files)>1 or bg_files:
  if fg_mode:
    with imageio.get_reader('/content/foreground', format='mp4') as reader:
      fps = reader.get_meta_data()['fps']
    if copy_audio:
      !ffmpeg -framerate $fps -i $out_dir/frame_%05d.png $fg_time_params -i /content/foreground -c:v libx264 -map 0:v -map 1:a? -vf "crop=trunc(iw/2)*2:trunc(ih/2)*2" -pix_fmt yuv420p /content/final.mp4 -y
    else:
      !ffmpeg -framerate $fps -i $out_dir/frame_%05d.png -c:v libx264 -vf "crop=trunc(iw/2)*2:trunc(ih/2)*2" -pix_fmt yuv420p /content/final.mp4 -y
  else:
    with imageio.get_reader('/content/background', format='mp4') as reader:
      fps = reader.get_meta_data()['fps']
    if copy_audio:
      !ffmpeg -framerate $fps -i $out_dir/frame_%05d.png $bg_time_params -i /content/background -c:v libx264 -map 0:v -map 1:a? -vf "crop=trunc(iw/2)*2:trunc(ih/2)*2" -pix_fmt yuv420p /content/final.mp4 -y
    else:
      !ffmpeg -framerate $fps -i $out_dir/frame_%05d.png -c:v libx264 -vf "crop=trunc(iw/2)*2:trunc(ih/2)*2" -pix_fmt yuv420p /content/final.mp4 -y
  #video can be downloaded from /content/final.mp4
  save_time = time()-start
  total_time = time()-grand_start  
  clear_output()
  with open('/content/final.mp4', 'rb') as f:
    data_url = "data:video/mp4;base64," + b64encode(f.read()).decode()
  display(HTML("""
  <video width=600 controls autoplay loop>
        <source src="%s" type="video/mp4">
  </video>""" % data_url))
else:
  shutil.move(out_dir+'/frame_%05d.png'%1, '/content/final.png')
  save_time = time()-start
  total_time = time()-grand_start
  clear_output()
  display(Image('/content/final.png', width=600))
if model.startswith('u2net'):
    print('frames=%i prepare=%i mask=%i blend=%i save=%i total=%i'%(len(iter_files), prepare_time, mask_time, blend_time, save_time, total_time))
else:
    print('frames=%i prepare=%i mask+blend=%i save=%i total=%i'%(len(iter_files), prepare_time, mask_time+blend_time, save_time, total_time))
  #image can be downloaded from /content/final.png

In [ ]:
#@title Download
#@markdown 1. If it fails try running this cell again.
#@markdown 2. Alternatively, you can manually download "final.mp4"/"final.png" from the folder on the left (click "Refresh" if missing).

print() #see https://github.com/googlecolab/colabtools/issues/468
from google.colab import files
if os.path.exists('/content/final.mp4'):
  files.download('/content/final.mp4') #fails for Firefox private window
else:
  files.download('/content/final.png')